In [1]:
from pymongo import MongoClient
from tqdm import tqdm
import numpy as np
import os
import pandas as pd
import pymongo
import xarray as xr

In [2]:
client = MongoClient()
db = client.eddies
collection = db.cchdo

In [3]:
def format(LON):
    if LON < 0:
        LON = LON + 360
    else:
        LON = LON
    return LON

In [4]:
rootdir = '/home/cz2397/data/cchdo-hyd/'

In [5]:
station = 0
overlap = 0
result = []

In [6]:
db.cchdo.create_index([('dat', pymongo.ASCENDING), ('loc', pymongo.ASCENDING)])

u'dat_1_loc_1'

In [7]:
for subdir, dirs, files in os.walk(rootdir):
    for file in tqdm(files):
        if file.endswith('hy1.nc'):
            url = os.path.join(subdir, file)
            DS = xr.open_dataset(url)
            DAT = DS.time.values[0]
            LAT = DS.latitude.values[0]
            LON = format(DS.longitude.values[0])
            dat_var = np.timedelta64(3, 'D')
            dat_min = pd.to_datetime(DAT - dat_var)
            dat_max = pd.to_datetime(DAT + dat_var)
            lat_var = 0.2
            lat_min = LAT - lat_var
            lat_max = LAT + lat_var
            lon_var = 0.4
            lon_min = LON - lon_var
            lon_max = LON + lon_var
            ft = {
                'dat': {'$gt': dat_min, '$lt': dat_max},
                'loc': {'$within': {'$box': [[lon_min, lat_min], [lon_max, lat_max]]}}
            }
            count = db.cchdo.find(ft).count()
            station = station + 1
            if count != 0:
                overlap = overlap + 1
                result.append(url)

100%|██████████| 152/152 [00:08<00:00, 16.92it/s]


In [8]:
station

39790

In [9]:
overlap

621

In [10]:
result.sort()

In [11]:
result

['/home/cz2397/data/cchdo-hyd/101_a24_nc_hyd/a24_00019_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/101_a24_nc_hyd/a24_00023_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/101_a24_nc_hyd/a24_00054_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/101_a24_nc_hyd/a24_00055_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/104_33KI131_1_nc_hyd/hot131_00002_00005_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/104_33KI131_1_nc_hyd/hot131_00002_00010_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/104_33KI131_1_nc_hyd/hot131_00002_00017_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/105_p17e_nc_hyd/p17e_00130_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/105_p17e_nc_hyd/p17e_00178_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/105_p17e_nc_hyd/p17e_00203_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/116_p02c_nc_hyd/p02c_00028_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/120_p14_49NZ20071008_nc_hyd/p01_49NZ20071008_00030_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/120_p14_49NZ20071008_nc_hyd/p01_49NZ20071008_00042_00001_hy1.nc',
 '/